In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
!pip install -q profanity-check
from profanity_check import predict

     |████████████████████████████████| 2.4MB 7.5MB/s 


/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imp

In [110]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')

In [111]:
count = data['Genre'].value_counts()
print(count.head(10))

Rock          121404
Pop           108714
Metal          20291
Jazz           13545
Folk            8644
Indie           8449
R&B             2793
Hip-Hop         2240
Electronic      2213
Country         1890
Name: Genre, dtype: int64


In [112]:
df = data[data['Language']=='en']
df

,Artist,Song,Genre,Language,Lyrics
0,12 stones,world so cold,Rock,en,"It starts with pain, followed by hate\nFueled ..."
1,12 stones,broken,Rock,en,Freedom!\nAlone again again alone\nPatiently w...
2,12 stones,3 leaf loser,Rock,en,"Biting the hand that feeds you, lying to the v..."
3,12 stones,anthem for the underdog,Rock,en,You say you know just who I am\nBut you can't ...
4,12 stones,adrenaline,Rock,en,My heart is beating faster can't control these...
...,...,...,...,...,...
290178,bobby womack,i wish he didn t trust me so much,R&B,en,I'm the best friend he's got I'd give him the ...
290179,bad boys blue,i totally miss you,Pop,en,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
290180,celine dion,sorry for love,Pop,en,Forgive me for the things That I never said to...
290181,dan bern,cure for aids,Indie,en,The day they found a cure for AIDS The day the...


In [113]:
options=['Metal']
df1 = data[data['Genre'].isin(options)]
df1 = pd.DataFrame(df, columns = ['Lyrics'])

In [114]:
df1

,Lyrics
0,"It starts with pain, followed by hate\nFueled ..."
1,Freedom!\nAlone again again alone\nPatiently w...
2,"Biting the hand that feeds you, lying to the v..."
3,You say you know just who I am\nBut you can't ...
4,My heart is beating faster can't control these...
...,...
290178,I'm the best friend he's got I'd give him the ...
290179,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
290180,Forgive me for the things That I never said to...
290181,The day they found a cure for AIDS The day the...


In [115]:
count=0
actual_count=0
for i in range(1000):
  #print(type(df1.iloc[i]['Lyrics']))
  if predict([df1.iloc[i]['Lyrics']])==1:
    #print("Offensive word found")
    count=count+1
    try:
      df1=df1.drop([i])
      actual_count=actual_count+1
    except:
      pass
  else:
    pass
print(len(df1))

249822


In [118]:
df1=df1.head(500)

In [119]:
len(df1)

500

In [120]:
metal_sub = df1.sample(n=500)
metal_sub.shape

(500, 1)

In [121]:
metal_sub

,Lyrics
396,Take a chance n something star you gotta\nAll ...
76,As he raises his hand you begin to understand\...
293,Yeah\nThere's a gravity in me\nPulling me to w...
720,"Listen, pick me up\nFill my cup\nPour me anoth..."
704,Well if you're lookin' for trouble\nI'm the ma...
...,...
524,"When the curtain falls it rolls, unwraps, remi..."
206,Put me on a train mama\n'cause things just ain...
296,"Awww s--t, what\nYou know the freaks can't han..."
717,"Who's your leader, who's your man?\nWho will h..."


In [122]:
metal_lyrics = metal_sub['Lyrics'].astype('str')


In [123]:
metallist = list(metal_lyrics)

metallist[0]

'Take a chance n something star you gotta\nAll the live long day\nIf I had to name one thing would data\nBe hard pressed to say\nYou can say that I’m gushy\nAt least you know what I think\nI know that I’m lucky I know you’re on the brink\nYou got it good good good you got it good so good\nYou make up ridiculous words\nWhich mean exactly nothing\nI know just what you mean and that’s the funny thing\nIt’s understood it’s understood\nBuddy buddy buddy I know why you wander\nThrough the sweetness on the ground\nThe sweetness of the flowers\nYou make your way on a cloudy day\nYou’ve got to move\nThe world you prove is too sweet yeah\nWe parted much too soon I know but that’s how it goes\nThere’s someone better out there for you and me\nCan’t wait to get on the road and feel the shows\nAnd be like phenomenon 1 and 2 and 3\nI don’t mean to brag and I don’t mean to boast\nBut I’m the guy could that give you the most\nIt’s good good good it’s understood\nChorus'

In [124]:
import string

def clean_text(txt):
    txt = txt.lower()
    txt = txt.replace("\n"," ")
    return txt 


In [125]:
corpus = [clean_text(x) for x in metallist]
corpus[0]

'take a chance n something star you gotta all the live long day if i had to name one thing would data be hard pressed to say you can say that i’m gushy at least you know what i think i know that i’m lucky i know you’re on the brink you got it good good good you got it good so good you make up ridiculous words which mean exactly nothing i know just what you mean and that’s the funny thing it’s understood it’s understood buddy buddy buddy i know why you wander through the sweetness on the ground the sweetness of the flowers you make your way on a cloudy day you’ve got to move the world you prove is too sweet yeah we parted much too soon i know but that’s how it goes there’s someone better out there for you and me can’t wait to get on the road and feel the shows and be like phenomenon 1 and 2 and 3 i don’t mean to brag and i don’t mean to boast but i’m the guy could that give you the most it’s good good good it’s understood chorus'

In [ ]:
len(corpus)

2500

In [126]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

In [127]:
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [128]:
total_words

6499

In [129]:
from keras.preprocessing.sequence import pad_sequences

max_sequence_len = 500

inp_sequences = np.array(pad_sequences(inp_sequences,
                       maxlen = max_sequence_len, padding='pre'))

In [17]:
print(inp_sequences.shape)
inp_sequences

(120228, 500)


array([[  0,   0,   0, ...,   0,  32,  32],
       [  0,   0,   0, ...,  32,  32, 287],
       [  0,   0,   0, ...,  32, 287,  36],
       ...,
       [  0,   0,   0, ...,   4,  78,  27],
       [  0,   0,   0, ...,  78,  27, 693],
       [  0,   0,   0, ...,  27, 693, 297]], dtype=int32)

In [130]:
predictors, label = inp_sequences[:,:-1],inp_sequences[:,-1]

In [131]:
import keras.utils as ku

label = ku.to_categorical(label, num_classes=total_words)

In [132]:
label.shape

(124312, 6499)

In [133]:
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding, LSTM
from keras.callbacks import EarlyStopping
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))
# Add a LSTM Layer
model.add(LSTM(100)) 
model.add(Dense(total_words/2, activation='relu'))  
# In the last layer, the shape should be equal to the total number of words present in our corpus
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics='accuracy')  
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 499, 50)           324950    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               60400     
_________________________________________________________________
dense_2 (Dense)              (None, 3249)              328149    
_________________________________________________________________
dense_3 (Dense)              (None, 6499)              21121750  
Total params: 21,835,249
Trainable params: 21,835,249
Non-trainable params: 0
_________________________________________________________________


In [134]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1)
mod_test = model.fit(predictors, label, epochs= 10, callbacks=[es])

Epoch 1/10
3885/3885 [==============================] - 124s 32ms/step - loss: 6.2267 - accuracy: 0.0514
Epoch 2/10
3885/3885 [==============================] - 121s 31ms/step - loss: 5.0396 - accuracy: 0.1428
Epoch 3/10
3885/3885 [==============================] - 121s 31ms/step - loss: 4.2374 - accuracy: 0.2236
Epoch 4/10
3885/3885 [==============================] - 121s 31ms/step - loss: 3.4893 - accuracy: 0.3173
Epoch 5/10
3885/3885 [==============================] - 122s 31ms/step - loss: 2.8695 - accuracy: 0.4046
Epoch 6/10
3885/3885 [==============================] - 121s 31ms/step - loss: 2.3234 - accuracy: 0.4920
Epoch 7/10
3885/3885 [==============================] - 121s 31ms/step - loss: 1.8665 - accuracy: 0.5729
Epoch 8/10
3885/3885 [==============================] - 121s 31ms/step - loss: 1.4462 - accuracy: 0.6561
Epoch 9/10
3885/3885 [==============================] - 121s 31ms/step - loss: 1.1214 - accuracy: 0.7269
Epoch 10/10
3885/3885 [==============================] 

In [135]:
!pip install -q pyyaml h5py

model.save('metal02.h5')

In [136]:
def make_lyrics(seed_text, next_words):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],
                     maxlen=max_sequence_len-1,padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    print(seed_text)

In [138]:
print (make_lyrics("light cool", 200))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


light cool the same sweatin in the rain so watch out there's a division in our life how do we take back what's been done what's been said oh my how do we take back 'cuz no one wins the wheel of them what you got so rude boy actin' coy you got nuts liek almond joy i tell ya this is a tribute i'm vexing many mofos but i'm wishing you come a day i didn't sit and mean so much can't hear about it stand up stand up stand up and face up and shakin’ it up now we just go yo dope rhymes might get crazy but they're not already there is not how it has it been since you let me in i need your heart and tears you laugh like then you think that i've done you don't mind yourself and if you dare nothing then waitings what i'll do what you mean screaming silent fears nothing don't try to be a prick to be a a a a a a a a little bit of what i'm a i will not attempt to try to be free but i reserve the right to be stoned i
None
